# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into deep neural networks for NLP!

## Objectives

You will be able to:

- Train a Word2Vec model and transform words into vectors 
- Obtain most similar words by using methods associated with word vectors 


## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec model. Then, you'll move onto building neural networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

Now, import the data. The data is stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a GitHub repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json()` function from Pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've imported the data, inspect the first few rows of the DataFrame to see what your data looks like. 

In [9]:
df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data -- `headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `'combined_text'` that consists of the data from the `'headline'` column plus a space character (`' '`) plus the data from the `'short_description'` column 
* Use the `'combined_text'` column's `.map()` method and pass in `word_tokenize`. Store the result returned in `data` 

In [10]:
df['combined_text'] = df['headline'] + ' ' + df['short_description']
data = df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [11]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from `gensim`. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1` 
    * The number of threads to use during training, `workers=4`

In [12]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)


Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call the `.train()` method on your model and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count` 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [14]:
corp_count = model.corpus_count

In [15]:
model.train(data,total_examples=corp_count,epochs=10)

(55585850, 67376200)

Great! You now have a fully trained model! The word vectors themselves are stored in the `Word2VecKeyedVectors` instance, which is stored in the `.wv` attribute. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [16]:
wv = model.wv


## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this by passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [18]:
wv.most_similar('Texas')

[('Maryland', 0.8189988136291504),
 ('Pennsylvania', 0.8097062706947327),
 ('Louisiana', 0.8066332936286926),
 ('Illinois', 0.7959516048431396),
 ('Utah', 0.7934251427650452),
 ('California', 0.7902024984359741),
 ('Oklahoma', 0.7871904373168945),
 ('Arkansas', 0.7861626148223877),
 ('Georgia', 0.7853074073791504),
 ('Massachusetts', 0.7797492742538452)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `.most_similar()` method's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [19]:
wv.most_similar(negative='Texas')

[('Per-Mile', 0.4187922477722168),
 ('Admon', 0.405057817697525),
 ('Headstrong', 0.39548245072364807),
 ('exiling', 0.39092379808425903),
 ('uh-oh', 0.3886106014251709),
 ('Animal-rescue', 0.3848515450954437),
 ('writer-producer', 0.3826277256011963),
 ('Uglies', 0.38062208890914917),
 ('Post-DOMA', 0.37113848328590393),
 ('Un-Blind', 0.3685277998447418)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [20]:
wv['Texas']

array([ 1.52401686e+00,  3.57077867e-02, -1.35766542e+00,  1.90347219e+00,
       -1.18290317e+00,  7.23412097e-01, -3.31274793e-03, -7.11298764e-01,
        1.31850362e-01, -1.98681390e+00,  1.69482017e+00,  8.46390873e-02,
       -1.37662455e-01, -1.55252844e-01,  2.60720015e-01, -2.03734565e+00,
        6.60532236e-01,  7.30779648e-01,  5.17582119e-01, -5.90889812e-01,
       -2.00614786e+00,  1.48710012e+00,  1.62612581e+00,  1.53105214e-01,
        2.26071283e-01, -7.22053289e-01,  3.43247104e+00,  7.33936191e-01,
       -1.68085396e+00,  2.53432453e-01,  3.14709496e+00, -1.66537988e+00,
       -1.85796082e+00,  4.79684174e-01,  1.29214895e+00, -1.45837843e+00,
       -4.13795352e-01, -2.64709722e-02,  8.23930085e-01,  3.57302403e+00,
       -2.42416239e+00,  5.69059253e-01,  2.03900203e-01, -1.97939306e-01,
        2.65418243e+00,  1.11263311e+00, -1.27970231e+00,  7.14184582e-01,
       -8.56697783e-02,  1.32680440e+00,  5.26041090e-01, -1.23494637e+00,
       -7.79690802e-01,  

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [21]:
wv.vectors

array([[ 1.7479864e-01,  1.2343022e+00, -4.3669754e-01, ...,
         7.9363042e-01,  2.1628965e-01, -1.5922369e+00],
       [-6.1775333e-01,  2.7868137e+00, -1.4269027e+00, ...,
         1.0401901e+00, -1.2035513e+00, -2.7606702e+00],
       [-1.6859987e+00,  2.2788405e+00, -9.8447013e-01, ...,
         1.1224650e+00,  1.5173489e-01, -2.5696988e+00],
       ...,
       [ 4.4320043e-02, -1.0340848e-01, -5.1692721e-02, ...,
        -1.5427366e-02, -1.1354334e-04,  4.2286418e-02],
       [ 2.0752439e-02,  3.0321063e-02, -5.9153706e-02, ...,
         7.3881410e-02, -3.4985449e-02, -3.4106988e-02],
       [ 3.8364671e-02, -6.6658489e-02, -2.0212371e-02, ...,
        -9.7160600e-03,  1.0829108e-02,  4.3775983e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `.most_similar()` method and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [25]:
wv.most_similar(positive=['king', 'woman'], negative=['man'])

[('ssy', 0.6191805601119995),
 ('depultures', 0.5634372234344482),
 ('NSYNC', 0.5618280172348022),
 ('glug', 0.5588288307189941),
 ('sl', 0.5586365461349487),
 ('tch', 0.550073504447937),
 ('ck', 0.5355126857757568),
 ('mp', 0.5272105932235718),
 ('Tr', 0.5253880620002747),
 ('tchiest', 0.525144100189209)]

As you can see from the output above, your model isn't perfect, but 'Queen' and 'Princess' are still in the top 5. As you can see from the other word in top 5, 'reminiscent' -- your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to create vectorized word embeddings!